In [1]:
import ibis
import pytest
from IPython import get_ipython

In [20]:
@pytest.fixture
def geo_con():
    mapd = ibis.mapd.connect(
        host='localhost', port='9091', 
        user='mapd', password='HyperInteractive', 
        database='ibis_testing'
    )
    
    print(mapd.con._client.get_version())
    
    return mapd


@pytest.fixture
def geo_table(geo_con):
    return geo_con.table('geo')


@pytest.fixture
def geo_point():
    point = (0, 1)
    return ibis.literal(point, type='point')


@pytest.fixture
def geo_linestring():
    linestring = [(0, 1), (2, 3)]
    return ibis.literal(linestring, type='linestring')


@pytest.fixture
def geo_polygon():
    polygon = [
        ((0, 0), (4, 0), (4, 4), (0, 4), (0, 0)),
        ((1, 1), (2, 1), (2, 2), (1, 2), (1, 1))
    ]
    return ibis.literal(polygon, type='polygon')


@pytest.fixture
def geo_multipolygon():
    polygon1 = (
        ((0, 0), (4, 0), (4, 4), (0, 4), (0, 0)),
        ((1, 1), (2, 1), (2, 2), (1, 2), (1, 1))
    )
    polygon2 = (
        ((10, 10), (14, 10), (14, 14), (10, 14), (10, 10)),
        ((11, 11), (12, 11), (12, 12), (11, 12), (11, 11))
    )
    multipolygon = [polygon1, polygon2]
    return ibis.literal(multipolygon, type='multipolygon')


def run_expr(table, func, arg=None):
    t = table
    f = func
    a = arg
    
    result = t[
        t.id,
        (f() if a is None else f(a)).name('tmp')
    ].sort_by('id').head(1).execute().loc[0, 'tmp']
    print(result)
    return result

def test_geo_op(
    geo_table, geo_point, geo_linestring, geo_polygon, geo_multipolygon
):
    """Testing for geo spatial operations.

    Parameters
    ----------
    geo_table : Ibis table with geo data
    geo_point : Geo spatial point
    geo_linestring : Geo spatial  line string
    geo_polygon : Geo spatial polygon
    geo_multipolygon : Geo spatial multi polygon

    """
    t = geo_table

    point = t.geo_point
    linestring = t.geo_linestring
    polygon = t.geo_polygon
    multipolygon = t.geo_multipolygon

    assert run_expr(t, point.max_distance, geo_linestring) == 3.6055512754639896
    assert run_expr(t, point.x) == 0
    assert run_expr(t, point.y) == 0
    assert run_expr(t, point.srid) == 0
    assert run_expr(t, linestring.x_min) == 0
    assert run_expr(t, linestring.x_max) == 1.0
    assert run_expr(t, polygon.y_min) == 0
    assert run_expr(t, polygon.y_max) == 2.0
    assert run_expr(t, multipolygon.n_points) == 12
    assert run_expr(t, multipolygon.n_rings) == 4
    
    expr = t[t, ibis.literal((0, 1), type="point:geometry").name('tmp')]
    print(expr.compile())
    expr.execute()
    #  ST_Distance
    #  ST_Contains
    #  ST_Area
    #  ST_Perimeter
    #  ST_Length
    #  ST_MaxDistance
    


In [21]:
if __name__ == '__main__':
    !ipython -m pytest -- IbisGeoSpatialFuncTests.ipynb -vv

]0;IPython: quansight/tmp============================= test session starts ==============================
platform linux -- Python 3.6.8, pytest-3.8.1, py-1.6.0, pluggy-0.7.1 -- /mnt/sda1/storage/miniconda/envs/ibis/bin/python
cachedir: .pytest_cache
rootdir: /mnt/sda1/dev/quansight/tmp, inifile:
plugins: xdist-1.23.2, forked-0.2, cov-2.6.0, importnb-0.5.5
collected 1 item                                                               

IbisGeoSpatialFuncTests.ipynb::test_geo_op FAILED                        [100%]

=================================== FAILURES ===================================
_________________________________ test_geo_op __________________________________

self = <pymapd.cursor.Cursor object at 0x7f54530607f0>
operation = "SELECT *, ST_GeomFromText('POINT(0 1)') AS tmp\nFROM geo\nLIMIT 10000"
parameters = None

    def execute(self, operation, parameters=None):
        # type: (str, tuple) -> Cursor
        """Execute a SQL statement.
    
            Parameters
  

In [4]:
print('SELECT *\nFROM (\n  SELECT "id", ST_STARTPOINT("geo_linestring") AS tmp\n  FROM geo\n) t0\nORDER BY "id"\nLIMIT 1')

SELECT *
FROM (
  SELECT "id", ST_STARTPOINT("geo_linestring") AS tmp
  FROM geo
) t0
ORDER BY "id"
LIMIT 1
